In [1]:
import sys
sys.path.insert(0,'../')

In [ ]:
import pandas as pd
import numpy as np
from utils import *
from datasets.fair_dataset import FairDataset
from IPython.display import display, Math

In [ ]:
class_shift = 10

dist = {'mus': {1: np.array([10, 13]), 
                0: np.array([10 - class_shift, 13 - class_shift])},
        'sigmas': [3, 3]
       }

In [ ]:
group_shift = 0
dist = {'mus':{'x1': {0: [0, 0 + group_shift], 1:[10, 10 + group_shift]},
               'z': [0, 2]},
        'sigmas': {'x1':{0: [5, 5], 1:[5, 5]},
                   'z': [1, 1]}
        }

In [ ]:
protected = ["sex"]
privileged_classes = [['Male']]
kwargs = {
    'protected_attribute_names': ['sex'],
    'privileged_group': 'Male',
    'favorable_class': 1,
    'classes': [0, 1],
    'sensitive_groups': ['Female', 'Male'],
    'group_shift': 0,
    'alpha': 0.5, 'beta': 1, 'keep_im_prot': False,
    'priv_ic_prob': 0.1, 'unpriv_ic_prob': 0.4,
    'dist': dist
}
kwargs

In [ ]:
estimator = get_estimator('nb', False)
keep_prot = False

In [ ]:
def get_performance_summary(train_fd, test_fd, estimator, prediction_func):
    pmod, pmod_results = get_groupwise_performance(train_fd, test_fd,
                                               estimator,
                                               privileged=True,
                                               pos_rate=False)
    umod, umod_results = get_groupwise_performance(train_fd, test_fd,
                                               estimator,
                                               privileged=False,
                                               pos_rate=False)
    mod, mod_results = get_groupwise_performance(train_fd, test_fd,
                                             estimator,
                                             privileged=None,
                                             pos_rate=False)

    p_perf = get_model_performances(pmod, test_fd,
                                prediction_func, keep_prot=keep_prot)
    u_perf = get_model_performances(umod, test_fd,
                                prediction_func, keep_prot=keep_prot)
    m_perf = get_model_performances(mod, test_fd,
                                prediction_func, keep_prot=keep_prot)
    print(get_table_row(is_header=False, var_value='mean', p_perf=p_perf,
                        u_perf=u_perf, m_perf=m_perf, variable='method'))
    return pmod, umod, mod, p_perf, u_perf, m_perf

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

def plot_normal(mu, sigma, ax, label=None):
    x = np.linspace(mu - 3*sigma, mu + 3*sigma, 100)
    ax.plot(x, stats.norm.pdf(x, mu, sigma), label=label)
    
def get_range(dist_table, i, j):
    mus = [l[j] for l in dist_table[:, :, 0].flatten()]
    sigmas = [l[j] for l in dist_table[:, :, 1].flatten()]
    vals = [mu - 3*sigmas[i] for i, mu in enumerate(mus)] 
    vals += [mu + 3*sigmas[i] for i, mu in enumerate(mus)]
    left = int(round(min(vals), 0))
    right = int(round(max(vals), 0))    
    return left, right
    
def plot_dist_table(dist_table, **kwargs):    
    f, ax = plt.subplots(2, 2)
    print(dist_table)
    # i controls sensitive attribute
    for i in range(2):
        # i controls class
        for j in range(2):
            mus = dist_table[i][j][0]
            sigmas = dist_table[i][j][1]
            plot_normal(float(mus[0]), float(sigmas[0]), ax[i][0])
            plot_normal(float(mus[1]), float(sigmas[1]), ax[i][1])

    for i in range(2):
        for j in range(2):
            left, right = get_range(dist_table, j, i)
            ax[j][i].set_xlim(left, right)
            ax[j][i].set_xticks([i for i in range(left, right+1) if i%5 == 0])
            ax[j][i].set_ylim(-0.01, 0.15)
            
    ax[0][0].set_title('Test Scores')
    ax[0][1].set_title('GPA')

    ax[1][0].set_ylabel('Privileged')
    ax[0][0].set_ylabel('Unprivileged')

    plt.subplots_adjust(wspace=0.25)

def plot_group_config(group_config):
    dist_table = np.reshape(group_config, (2,2,4))
    plot_dist_table(dist_table)

Test plot_dist_table()   

In [ ]:
group_config = [
    ([5, 0], [3, 3], 0, 0),
    ([5, 10], [3, 3], 0, 1),
    ([0, 5], [3, 3], 1, 0),
    ([10, 5], [3, 3], 1, 1)
]
plot_group_config(group_config)

In [ ]:
train_fd, test_fd = get_synthetic_train_test_split(
    type='corr', n_samples=10000, n_features=2,
    train_random_state=47, test_random_state=41,
    method='simple_imputer.mean', test_method='train',
    **kwargs)

In [ ]:
complete_df = train_fd.complete_df.astype(float)

In [ ]:
for s in [0, 1]:
    for y in [0, 1]:
        sub_df = complete_df[(complete_df['sex'] == s) & (complete_df['label'] == y)]
        print(sub_df.describe())
        
for y in [0, 1]:
    sub_df = complete_df[(complete_df['label'] == y)]
    print(sub_df.describe())

In [ ]:
train_fd.group_configs

In [ ]:
np.reshape(train_fd.group_configs, (2,2,4))

In [ ]:
plot_group_config(train_fd.group_configs)

In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(complete_df[complete_df.columns[:-2]], complete_df[complete_df.columns[-1]])
model.theta_

In [ ]:
model.classes_

In [ ]:
model.var_

In [ ]:
def get_df_group_config(df, precision=3):
    df_gc = []
    
    for s in [0, 1]:
        for l in [0, 1]:
            desc = df[((df['sex'] == s) & (df['label'] == l))].describe()
            mus = desc.loc['mean']
            sigmas = desc.loc['std']
            df_gc.append((
                [round(mus[0], precision), round(mus[1], precision)], 
                [round(sigmas[0], precision), round(sigmas[1], precision)], s, l)
            ) 
    print(*df_gc, sep='\n')
    return df_gc

In [ ]:
group_config = get_df_group_config(complete_df)
plot_group_config(group_config)

In [ ]:
incomplete_df = train_fd.get_incomplete_df()
group_config = get_df_group_config(incomplete_df)
plot_group_config(group_config)

In [ ]:
incomplete_df.describe()

In [ ]:
imputed_df = train_fd.imputed_df
group_config = get_df_group_config(imputed_df)
plot_group_config(group_config)
print(complete_df.describe())
print(imputed_df.describe())

In [ ]:
get_df_group_config(imputed_df)
get_df_group_config(complete_df)

In [ ]:
complete_df[complete_df['sex'] == 0].corr()

In [ ]:
complete_df[complete_df['sex'] == 1].corr()

In [ ]:
imputed_df[imputed_df['sex'] == 0].corr()

In [ ]:
imputed_df[imputed_df['sex'] == 1].corr()

In [ ]:
print(all(test_fd.complete_df == test_fd.imputed_df))
test_df = test_fd.imputed_df
get_df_group_config(test_df)

In [ ]:
print(get_table_row(is_header=True, variable='Method'))
# output = get_performance_summary(baseline_fd, baseline_fd, estimator, get_predictions)
output = get_performance_summary(train_fd, test_fd, estimator, get_predictions)

In [ ]:
pmod, umod, mod, _, _, _ = output

In [ ]:
pmod.theta_

In [ ]:
pmod.var_

In [ ]:
umod.theta_

In [ ]:
umod.var_

In [ ]:
mod.theta_

In [ ]:
mod.var_

In [ ]:
group_configs = get_df_group_config(test_fd.imputed_df)
plot_group_config(group_configs)
figure = plt.gcf()
axes = figure.get_axes()
means = np.mean(mod.theta_, axis=0)
print(means)
print(axes)
for i in [0, 1]:
    for j in [0, 1]:
        print(means[i])
        axes[2*i + j].axvline(means[j])

Gamma is complete case probability

In [ ]:
alpha = 0.25
beta = 1
alpha_y = np.array([[alpha], [1-alpha]])
prob_s = np.array([1/(1+beta), beta/(1+beta)])
gamma_s = np.array([0.9, 0.6])

In [ ]:
mu = np.array([[[1, 2], [11, 12]],
               [[0, 3], [10, 13]]])

var = np.array([[[3, 3], [3, 3]],
                [[3, 3], [3, 3]]])

In [ ]:
mu_plus = (prob_s * mu[:, 1]).sum(axis=0)
mu_minus = (prob_s * mu[:, 0]).sum(axis=0)
print(mu_plus, mu_minus)

In [ ]:
mu_s = (alpha_y * mu).sum(axis=1)
print(mu_s)

$\newcommand{\bvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\redtext}[1]{\color{red}#1\color{black}}
\newcommand{\numberthis}[0]{\stepcounter{equation}\tag{\theequation}}
\newcommand{\mynorm}[1]{\mid \mid #1 \mid \mid}
\newcommand{\dep}{\perp \!\!\! \perp}
\newcommand{\indep}{\centernot{\dep}}
\newcommand{\dsP}{P}
\newcommand{\dsE}{E}$
\begin{align*}
\dsE[\bvec{x} | \bvec{r} = \bar{0}] =& \sum_{s}{\dfrac{\gamma_s\dsP(s)}{\sum_{s}{\gamma_s \dsP(s)}}\mu^{s}}\\
\end{align*}


In [ ]:
# mu_impute = ((prob_s * gamma_s * mu_s)/ (prob_s * gamma_s).sum()).sum()
mu_impute = gamma_s * mu_s
print('mu_impute', mu_impute)

mu_s_imputed = (prob_s * gamma_s) * mu_s + prob_s * (1 - gamma_s) * mu_impute
print('mu_s_imputed', mu_s_imputed)
mu_dash = mu_s_imputed.sum()
print('mu_dash', mu_dash)
assert mu_impute == mu_dash

\begin{align*}
\dsE[\bvec{x}^\prime \mid s, y] &= \sum_{\bvec{r}}{\dsP(\bvec{r} \mid s, y)\dsE[\bvec{x}^\prime \mid \bvec{r}, s, y]}\\
&= \left[\dsP(\bvec{r}=\bar{0}|s) \dsE[\bvec{x}' \mid s, y, \bvec{r}=\bar{0}] + \dsP(\bvec{r} \neq 0 | s) \dsE[\bvec{x}' \mid s, y, \bvec{r} \neq 0]\right]\\
&= \gamma_s \dsE[\bvec{x} \mid s, y] + (1-\gamma_s) \mu_{\bvec{r}=0}\\
\end{align*}


In [ ]:
display(Math(r'$E[x^\prime | s, +]$'))
mu_prime_s_plus = gamma_s * mu_plus + (1-gamma_s) * mu_impute
print(mu_prime_s_plus)

display(Math(r'$E[x^\prime | s, -]$'))
mu_prime_s_minus = gamma_s * mu_minus + (1-gamma_s) * mu_impute
print(mu_prime_s_minus)

\begin{align*}
E[\bvec{x}^{\prime} \mid y] =& \sum_s{P(s) E[\bvec{x}^{\prime} \mid s, y]}\\
=& P(p) E[\bvec{x}^{\prime} \mid p, y] + P(u) E[\bvec{x}^{\prime} \mid u, y]\\
=& \dfrac{\beta}{1+\beta} E[\bvec{x}^{\prime} \mid p, y] + \dfrac{1}{1+\beta} E[\bvec{x}^{\prime} \mid u, y]
\end{align*}

In [ ]:
prob_s * mu_prime_s_plus + (1-prob_s) * mu_prime_s_minus

In [ ]:
# group_config = get_df_group_config(imputed_df)
# plot_group_config(group_config)
# plt.gca()
# mus = mod.theta_.mean(axis=0)
# decision_mus = np.array([mus, mus]).flatten()
# for a, x_line in zip(plt.gcf().get_axes(), decision_mus):
#     a.vlines(x_line, -1, 1)

In [ ]:
get_df_group_config(imputed_df)

In [ ]:
group_config = [
    ([0, 0], [3, 3], 0, 0),
    ([0, 0], [3, 3], 1, 0),
    ([10, 13], [3, 3], 0, 1),
    ([10, 13], [3, 3], 1, 1)
]
group_config = get_df_group_config(imputed_df)
plot_group_config(group_config)
plt.gca()
mus = mod.theta_.mean(axis=0)
decision_mus = np.array([mus, mus]).flatten()
for a, x_line in zip(plt.gcf().get_axes(), decision_mus):
    a.vlines(x_line, -1, 1)

Test data

In [ ]:
alpha = 0.5
beta = 1
alpha_y = np.array([[alpha], [1-alpha]])
prob_s = np.array([1/(1+beta), beta/(1+beta)])
gamma_s = np.array([1, 0.6])

In [ ]:
df = pd.read_csv('data.tsv', sep='\t')
print(df)

In [ ]:
def get_column_stats(df, column):
    statistics = []
    for (s, y), grp in df.groupby(['s', 'y']):
        statistics.append([s, y, grp[column].mean(), grp[column].var()])

    for s, grp in df.groupby(['s']):
        statistics.append([s, -1, grp[column].mean(), grp[column].var()])

    for y, grp in df.groupby(['y']):
        statistics.append([-1, y, grp[column].mean(), grp[column].var()])

    statistics.append([-1, -1, df[column].mean(), df[column].var()])

    statistics = pd.DataFrame(statistics, columns=['s', 'y', 'mean', 'var'])
    return statistics

In [ ]:
orig_stats = get_column_stats(df, 'x_orig')
orig_stats

$\newcommand{\bvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\redtext}[1]{\color{red}#1\color{black}}
\newcommand{\numberthis}[0]{\stepcounter{equation}\tag{\theequation}}
\newcommand{\mynorm}[1]{\mid \mid #1 \mid \mid}
\newcommand{\dep}{\perp \!\!\! \perp}
\newcommand{\indep}{\centernot{\dep}}
\newcommand{\dsP}{P}
\newcommand{\dsE}{E}$
\begin{align*}
\dsE[\bvec{x} | \bvec{r} = \bar{0}] =& \sum_{s}{\dfrac{\gamma_s\dsP(s)}{\sum_{s}{\gamma_s \dsP(s)}}\mu^{s}}\\
\end{align*}


In [ ]:
orig_stats_s = orig_stats[(orig_stats['s'] != -1) & (orig_stats['y'] == -1)]
mu_impute = (gamma_s * prob_s * orig_stats_s['mean'].values / (gamma_s * prob_s).sum()).sum()
mu_impute

In [ ]:
get_column_stats(df, 'x_miss')

Random

In [31]:
mu_p_1 = 13
mu_p_0 = 3
mu_u_1 = 10
mu_u_0 = 0

In [33]:
gamma_p = 0.9
gamma_u = 0.7
alpha = 0.5

In [37]:
mu_p = alpha * mu_p_1 + (1-alpha)*mu_p_0
mu_u = alpha * mu_u_1 + (1-alpha)*mu_u_0

In [38]:
mu_p, mu_u

(8.0, 5.0)

In [39]:
mu_m = (gamma_p * mu_p + gamma_u * mu_u) / (gamma_p + gamma_u)

In [42]:
term_1 = (mu_p_1 + mu_u_1) - 2*mu_m 

In [44]:
term_2 = -(gamma_p * mu_p_1 + gamma_u * mu_u_1) + (gamma_p + gamma_u) * mu_m

In [46]:
term_1 > term_2

True

In [47]:
mu_1_prime = 0.5*(gamma_p * mu_p_1 + gamma_u * mu_u_1) + (1 - (gamma_p + gamma_u)/ 2) * mu_m

In [48]:
mu_1_prime

10.6875

In [51]:
mu_p_1_prime = gamma_p * mu_p_1 + (1-gamma_p) * mu_m
mu_p_0_prime = gamma_p * mu_p_0 + (1-gamma_p) * mu_m

In [53]:
mu_p_1_prime, mu_p_0_prime

(12.36875, 3.36875)

In [54]:
mu_u_1_prime = gamma_u * mu_u_1 + (1-gamma_u) * mu_m
mu_u_0_prime = gamma_u * mu_u_0 + (1-gamma_u) * mu_m

In [55]:
mu_u_1_prime, mu_u_0_prime

(9.00625, 2.00625)

In [56]:
mu_1_prime = (gamma_p * mu_p_1_prime + gamma_u * mu_u_1_prime)/(gamma_p + gamma_u) 

In [57]:
mu_1_prime

10.89765625

In [58]:
mu_0_prime = (gamma_p * mu_p_0_prime + gamma_u * mu_u_0_prime)/(gamma_p + gamma_u) 

In [59]:
mu_0_prime

2.7726562499999994

In [60]:
(mu_p_1 - mu_1_prime)**2

4.41984924316406

In [61]:
(mu_u_1 - mu_1_prime)**2

0.8057867431640637